In [ ]:
#notebook cell to generate cell images for all sample slices
import os
import numpy as np
import csv
from cv2 import imread
from microscope_image_preprocessing import test_threshold, cut_image
from square_cut import square_cut_save
from active_thresh_segseg_label import segseg_label

PATH = 'D:\\ilham_skynet\\Bact_photo'
PARAM_PATH = 'params.txt'
super_dir = os.listdir(PATH)
num=0

for dir in super_dir:
    dir_path = PATH + '\\' + dir
    filelist = [os.path.join(dir_path,x) for x in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path,x))]
    for filepath in filelist:
        cuts_list = [os.path.join(filepath,x) for x in os.listdir(filepath) if x.endswith('.jpg')]
        for cuts in cuts_list:
            # print(cuts)
            segseg_label(cuts)
    


In [21]:
#notebook cell to generate labels in csv format
#iterates through each directory and creates a summary of the contents of it's subdirectories in csv format
import os
import numpy as np
import csv
from cv2 import imread
from microscope_image_preprocessing import test_threshold, cut_image
from square_cut import square_cut_save
from active_thresh_segseg_label import segseg_label
from labelfile import ret_model, load_dir, ret_predict

PATH = 'D:\\ilham_skynet\\Bact_photo'
PARAM_PATH = 'params.txt'
super_dir = os.listdir(PATH)
num=0
total = []

model = ret_model('enc10_3.keras')

for dir in super_dir:
    dir_path = PATH + '\\' + dir
    filelist = [os.path.join(dir_path,x) for x in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path,x))]
    per_set = []
    for filepath in filelist:
        cuts_list = [os.path.join(filepath,x) for x in os.listdir(filepath) if not x.endswith('.jpg') and 'label' not in x]
        print(filepath)
        per_cut = []
        for cuts in cuts_list:
            print(cuts)
            mini_names, img_gen = load_dir(cuts)
            labels = ret_predict(model, img_gen, len(mini_names))
            per_image = list(zip(mini_names,labels))
            print(per_image)

            with open(cuts + '\\labels.csv', 'w') as csv_file:
                fieldnames = ['filename', 'label']
                csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                for name, label in per_image:
                    csv_writer.writerow({'filename':name, 'label':label})
            csv_file.close()

            per_cut += per_image

        with open(filepath + '\\labels.csv', 'w') as csv_file:
            fieldnames = ['filename', 'label']
            csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            for name, label in per_cut:
                csv_writer.writerow({'filename':name, 'label':label})
        csv_file.close()
        per_set += per_cut

    with open(dir_path + '\\labels.csv', 'w') as csv_file:
            fieldnames = ['filename', 'label']
            csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            for name, label in per_set:
                csv_writer.writerow({'filename':name, 'label':label})
    csv_file.close()
    total += per_set

with open(PATH + '\\labels.csv', 'w') as csv_file:
            fieldnames = ['filename', 'label']
            csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            for name, label in total:
                csv_writer.writerow({'filename':name, 'label':label})

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 32)   320         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 32, 32)   9248        ['conv2d[0][0]']                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 16, 16, 32)   0           ['conv2d_1[0][0]']               
                                                                                              